In [1]:
import tensorflow as tf
import numpy as np
import cv2
import matplotlib.pyplot as plt

# load a pre-trained maskRcnn model
model = tf.keras.applications.ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

model = tf.keras.models.load_model('path_to_mask_rcnn_model.h5')

print("Model loaded")


94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 7s 0us/step


FileNotFoundError: [Errno 2] Unable to synchronously open file (unable to open file: name = 'path_to_mask_rcnn_model.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

This section below is for testing with a few imgs. load filename in placeholder

In [ ]:
def load_image_into_numpy_array(path):
    """Load an image from file into a numpy array."""
    image = cv2.imread(path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    return image

# placeholder 
image_path = 'path_to_your_image.jpg' 
image_np = load_image_into_numpy_array(image_path)

# resize img
image_resized = cv2.resize(image_np, (1024, 1024))  # maskRcnn uses 1024x1024 images

# display inp img
plt.figure(figsize=(10, 10))
plt.imshow(image_resized)
plt.axis('off')
plt.show()


In [ ]:
# Detection

# img to batch size 1
input_tensor = tf.convert_to_tensor(image_resized)
input_tensor = input_tensor[tf.newaxis, ...]

detections = model(input_tensor)

# extract detection results - boxes masks classes scores
# may need adjustments
detection_boxes = detections['detection_boxes'][0].numpy()
detection_scores = detections['detection_scores'][0].numpy()
detection_classes = detections['detection_classes'][0].numpy()
detection_masks = detections['detection_masks'][0].numpy()

print("Detection done")


In [ ]:
def display_detections(image_np, boxes, masks, classes, scores, threshold=0.5):
    """Display the detection results on the image."""
    for i in range(min(len(boxes), len(scores))):
        if scores[i] > threshold:
            # extract bounding box
            box = boxes[i]
            ymin, xmin, ymax, xmax = box

            # draw bounding box
            cv2.rectangle(image_np, (int(xmin), int(ymin)), (int(xmax), int(ymax)), (0, 255, 0), 2)

            # class label w confidence score
            label = f"Class {int(classes[i])}: {scores[i]:.2f}"
            cv2.putText(image_np, label, (int(xmin), int(ymin) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

            # draw masks if supported idk
            mask = masks[i]
            mask = cv2.resize(mask, (image_np.shape[1], image_np.shape[0]))
            image_np[mask > 0.5] = (0, 255, 0)  # Color the mask area green

    # show img
    plt.figure(figsize=(10, 10))
    plt.imshow(image_np)
    plt.axis('off')
    plt.show()

# show detections
display_detections(image_resized, detection_boxes, detection_masks, detection_classes, detection_scores)
